In [ ]:
import numpy as np
import pywt
from PIL import Image
import os
import pickle
from decimal import Decimal, getcontext
from collections import defaultdict
import time
import csv



✅ Duration tracking complete. Saved to: dwt_arithmetic.csv


Arithmetic Coding

In [ ]:
getcontext().prec = 200
BLOCK_SIZE = 32  # Block size for Arithmetic coding

class ArithmeticEncoder:
    def __init__(self, freq):
        self.freq = freq
        self.total = sum(freq.values())
        self.cum_freq = self.build_cum_freq()
        self.low = Decimal(0)
        self.high = Decimal(1)

    def build_cum_freq(self):
        cum_freq = {}
        total = 0
        for symbol in range(65536):  # 16-bit range
            cum_freq[symbol] = total
            total += self.freq.get(symbol, 0)
        return cum_freq

    def encode(self, data):
        self.low = Decimal(0)
        self.high = Decimal(1)
        for symbol in data:
            range_ = self.high - self.low
            low_count = self.cum_freq[symbol]
            high_count = low_count + self.freq[symbol]
            self.high = self.low + range_ * Decimal(high_count) / self.total
            self.low = self.low + range_ * Decimal(low_count) / self.total
        return (self.low + self.high) / 2

class ArithmeticDecoder:
    def __init__(self, freq, encoded_value, data_length):
        self.low = Decimal(0)
        self.high = Decimal(1)
        self.value = encoded_value
        self.freq = freq
        self.total = sum(freq.values())
        self.cum_freq = self.build_cum_freq()
        self.symbols = list(range(65536))
        self.data_length = data_length

    def build_cum_freq(self):
        cum_freq = {}
        total = 0
        for symbol in range(65536):
            cum_freq[symbol] = total
            total += self.freq.get(symbol, 0)
        return cum_freq

    def decode(self, length):
        decoded = []
        for _ in range(length):
            range_ = self.high - self.low
            if range_ == 0:
                decoded.append(max(self.freq.items(), key=lambda x: x[1])[0])
                continue
            scaled_value = (self.value - self.low) * self.total / range_
            found = False
            for symbol in self.symbols:
                low_count = self.cum_freq[symbol]
                high_count = low_count + self.freq[symbol]
                if low_count <= scaled_value < high_count:
                    decoded.append(symbol)
                    self.high = self.low + range_ * Decimal(high_count) / self.total
                    self.low = self.low + range_ * Decimal(low_count) / self.total
                    found = True
                    break
            if not found:
                decoded.append(0)
        return decoded



DWT

In [ ]:

def compress_ecg_image(img_array, wavelet='haar', level=2, threshold=10):
    coeffs = pywt.wavedec2(img_array, wavelet=wavelet, level=level)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    coeff_arr[np.abs(coeff_arr) < threshold] = 0
    return coeff_arr, coeff_slices

def decompress_ecg_image(coeff_arr, coeff_slices, wavelet='haar'):
    coeffs_from_arr = pywt.array_to_coeffs(coeff_arr, coeff_slices, output_format='wavedec2')
    reconstructed = pywt.waverec2(coeffs_from_arr, wavelet=wavelet)
    return np.clip(reconstructed, 0, 255).astype(np.uint8)



In [ ]:

durations = []
def compress_and_reconstruct_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        start_time = time.time()
        input_path = os.path.join(input_folder, filename)
        image = Image.open(input_path).convert('L')
        arr = np.array(image)

        # Compression
        coeff_arr, coeff_slices = compress_ecg_image(arr)
        quantized = np.round(coeff_arr).astype(np.int16)
        flat = quantized.flatten()

        # Prepare 16-bit unsigned values
        unsigned_flat = np.array([val & 0xFFFF for val in flat], dtype=np.uint16)
        flat_list = unsigned_flat.tolist()

        # Build frequency table
        freq = defaultdict(int)
        for i in range(65536):
            freq[i] = 1  # Laplace smoothing
        for val in flat_list:
            freq[val] += 1

        # Encode
        encoder = ArithmeticEncoder(freq)
        blocks = [flat_list[i:i+BLOCK_SIZE] for i in range(0, len(flat_list), BLOCK_SIZE)]
        encoded_blocks = [str(encoder.encode(block)) for block in blocks]
        block_sizes = [len(block) for block in blocks]

        # Save compressed data
        compressed_data_path = os.path.join(output_folder, filename + '.bin')
        with open(compressed_data_path, 'wb') as f:
            pickle.dump((encoded_blocks, block_sizes, freq, quantized.shape, coeff_slices), f)

        # Decompression
        with open(compressed_data_path, 'rb') as f:
            encoded_blocks, block_sizes, freq, shape, coeff_slices = pickle.load(f)

        decoded_flat = []
        for block_val_str, block_size in zip(encoded_blocks, block_sizes):
            encoded_val = Decimal(block_val_str)
            decoder = ArithmeticDecoder(freq, encoded_val, block_size)
            decoded_block = decoder.decode(block_size)
            decoded_flat.extend(decoded_block)

        decoded_signed = np.array([((val + 2**15) % 2**16) - 2**15 for val in decoded_flat], dtype=np.int16)

        reshaped = decoded_signed.reshape(shape)
        reconstructed = decompress_ecg_image(reshaped, coeff_slices)

        # Save reconstructed image
        save_path = os.path.join(output_folder, filename)
        Image.fromarray(reconstructed).save(save_path)
        end_time = time.time()
        duration = end_time - start_time
        durations.append((filename, duration))
        
    csv_path = 'dwt_arithmetic.csv'
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Image Name', 'Duration (seconds)'])
        writer.writerows(durations)
        
        print(f"✅ Duration tracking complete. Saved to: {csv_path}")


#main function to process
compress_and_reconstruct_folder("org_ecg_10sec_resize", "after compress dwt arithmetic")